## Scraping Tweets

Consider using Twint? https://github.com/twintproject/twint

Fields from Twint : [‘id’, ‘conversation_id’, ‘created_at’, ‘date’, 
         ‘timezone’,    ‘place’, ‘tweet’, ‘hashtags’, ‘cashtags’,
         ‘user_id’, ‘user_id_str’, ‘username’, ‘name’, ‘day’,
         ‘hour’, ‘link’, ‘retweet’, ‘nlikes’, ‘nreplies’,
         ‘nretweets’, ‘quote_url’, ‘search’, ‘near’, ‘geo’,
         ‘source’, ‘user_rt_id’, ‘user_rt’, ‘retweet_id’,
         ‘reply_to’, ‘retweet_date’, ‘translate’, ‘trans_src’,
         ‘trans_dest’]

In [1]:
import tweepy
import datetime
import json
import sys

In [4]:
API_key_secret = "..."
API_key = "..."
access_token = "..."
access_token_secret = "..."

auth = tweepy.OAuthHandler(API_key, API_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit_notify=True)

tweets_per_file = 50000
count = 0

In [5]:
query = () 
lat = 
long = 
radius = 

In [10]:
class Listener(tweepy.StreamListener):
    def __init__(self, output_file=sys.stdout):
        super(Listener,self).__init__()
        self.tweets = []
    
    def on_status(self, status): #if want to search text then put if not contains x then continue
        self.tweets.append({
                       "id": status.id,
                       "text": status.text,
                       "hashtags": [d['text'] for d in status.entities['hashtags']],
                       "username": status.user.name,
                       "timestamp": status.created_at.strftime('%H:%M %d/%m/%Y'),
                       "location": status.coordinates["coordinates"] if status.coordinates is not None
                         else '', 
                       "replied_to": status.in_reply_to_screen_name if status.in_reply_to_screen_name
                         is not None else '',
                       "language": status.lang, 
                       "verified": status.user.verified,
                       "followers": status.user.followers_count,
                       "friends": status.user.friends_count,
                       "statuses": status.user.statuses_count, 
                       "retweeted": status.retweet_count})
    
    def on_error(self, status_code):
        print("Error", status_code)
        return True # dont stop
        print("Stream restarted")

    def on_timeout(self):
        print >> sys.stderr, 'Timeout...'
        return True # don't stop
        print("Stream restarted")

In [11]:
def save_tweets(tweets, count, count_tweets, tweets_per_file):
    file_name = int(count)
    temp = count*tweets_per_file
    count_tweets = temp+tweets_per_file
    if len(tweets)>= count_tweets:
        obj = tweets[temp:(temp+tweets_per_file)]
        with open(f'output_listen_amazon_{file_name}.json', 'w') as fout:#set indent to 0 if don't need human readable
            json.dump(obj, fout, indent=4)
    elif len(tweets)< count_tweets:
        obj = tweets[temp:]
        with open(f'output_listen_amazon_{file_name}.json', 'w') as fout:
            json.dump(obj, fout, indent=4)
    if len(tweets)> count_tweets:
        save_tweets(tweets, count+1, count_tweets, tweets_per_file)

In [12]:
listener = Listener()

stream = tweepy.Stream(auth=api.auth, listener=listener)
try:
    print('Start streaming.')
    stream.filter(languages=["en"], track=[query]) #stream.filter(languages=["en"], locations=[-6.38,49.87,1.77,55.81])

except KeyboardInterrupt as e :
    print("Stopped")

finally:
    print('Done.')
    stream.disconnect()
    save_tweets(listener.tweets,count,0,tweets_per_file)


Start streaming.
Done.


ReadTimeoutError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.